In [1]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the numpy and pandas packages

import numpy as np
import pandas as pd
import codecs as cd

Load the companies and rounds data (provided on the previous page) into two data frames and name them companies and rounds2 respectively.

Import and read the companies text file, funding round details csv. Store it in variables called companies and rounds2.

In [3]:
#save the companies.txt file as proper UTF-8 format before loading in dataframe
encodedFile = 'companies_encoded.txt'
sourceEncoding = "utf-8"
targetEncoding = "utf-8"
source = open("companies.txt",errors='ignore')
target = open(encodedFile, "wb")

target.write(str(source.read()).encode(targetEncoding))
companies = pd.read_csv(encodedFile, sep='\t',encoding='UTF-8') # read the txt using read_csv command.
companies.head()
print(companies.shape) # (66368 rows, 10columns)

(66368, 10)


In [4]:
#save the rounds2.csv file as proper UTF-8 format before loading in dataframe
encodedFile = 'rounds2_encoded.csv'
sourceEncoding = "utf-8"
targetEncoding = "utf-8"
source = open("rounds2.csv",errors='ignore')
target = open(encodedFile, "wb")

target.write(str(source.read()).encode(targetEncoding))
rounds2 = pd.read_csv(encodedFile,encoding='UTF-8') # read the txt using read_csv command.
rounds2.head()
print(rounds2.shape) #114949 rows, 6 columns

(114949, 6)


In [21]:
#Table 1.1, Question 1: How many unique companies are present in rounds 2
rounds2['company_permalink'].nunique()
rounds2.head()
rounds2['company_permalink'].str.lower().nunique()

66368

In [25]:
#Table 1.1, Question 2: How many unique companies are present in companies
companies.head()
companies['permalink'].str.lower().nunique()

66368

In [106]:
#Table 1.1, Question 3: In the companies data frame, which column can be used as the  unique key for each company? Write the name of the column.
#Answer - permalink column

In [26]:
#Table 1.1, Question 4: Are there any companies in the rounds2 file which are not  present in companies ? Answer Y/N.
# compare the two columns to get the companies absent in the companies DF
rounds2['is_absent_inCompanies'] = rounds2['company_permalink'].str.lower().isin(companies['permalink'].str.lower()).astype(np.int8)
AbsentCompanies = rounds2.loc[rounds2.is_absent_inCompanies.apply(lambda x: x==0)]
rounds2 = rounds2.drop('is_absent_inCompanies',axis=1)
AbsentCompanies.head()
print(AbsentCompanies.shape) #0 companies are there in rounds 2 which are absent in companies


(0, 7)


In [9]:
#rounds2.loc[~rounds2['company_permalink'].isin(companies['permalink']), :]
print(len(rounds2.loc[~rounds2['company_permalink'].isin(companies['permalink']), :]))
#print(len(rounds2.loc[companies['permalink']]))

114949


In [27]:
#Table 1.1, Question 5:Merge the two data frames so that all  variables (columns)  in the companies frame are added to the rounds2 data frame. Name the merged frame master_frame.
#lowering the column names and column values before joining
rounds2.rename(columns=lambda x: x.lower(), inplace=True)
companies.rename(columns=lambda x: x.lower(), inplace=True)
rounds2['company_permalink']=rounds2['company_permalink'].str.lower()
companies['permalink']=companies['permalink'].str.lower()
master_frame = pd.merge(rounds2, companies,  how='left', left_on='company_permalink', right_on = 'permalink')
master_frame.head(10)
print(master_frame.shape) #114949

(114949, 16)


In [28]:
#Table 2.1 Average Values of Investments for Each of these Funding Types
#Group by funding type
# Apply mean() on each of fundingTypes
by_funding_type = master_frame.groupby('funding_round_type')
by_funding_type['raised_amount_usd'].mean() #Average funding amount of each funding Type

funding_round_type
angel                    9.586945e+05
convertible_note         1.453439e+06
debt_financing           1.704353e+07
equity_crowdfunding      5.383682e+05
grant                    4.300576e+06
non_equity_assistance    4.112031e+05
post_ipo_debt            1.687046e+08
post_ipo_equity          8.218249e+07
private_equity           7.330859e+07
product_crowdfunding     1.363131e+06
secondary_market         7.964963e+07
seed                     7.198180e+05
undisclosed              1.924237e+07
venture                  1.174895e+07
Name: raised_amount_usd, dtype: float64

In [13]:
by_funding_type['raised_amount_usd'].count() # To check where most investors have invested, which turns out as venture type

funding_round_type
angel                     4860
convertible_note          1447
debt_financing            6682
equity_crowdfunding       1228
grant                     2124
non_equity_assistance       73
post_ipo_debt              152
post_ipo_equity            628
private_equity            1936
product_crowdfunding       386
secondary_market            30
seed                     23603
undisclosed               1582
venture                  50228
Name: raised_amount_usd, dtype: int64

In [29]:
#Table 3.1 top nine countries which have received the highest total funding (across ALL sectors for the chosen investment type)
get_venture_investmentType_group= by_funding_type.get_group('venture')# use previous dataframe group by_funding_type to get only venture types
by_country_type = get_venture_investmentType_group.groupby(['country_code']) # group the result by country code
topcountries_withfunding=by_country_type['raised_amount_usd'].sum().sort_values(ascending=False).head(9) # get total investment amount and sort by 9 max countries
top9 = topcountries_withfunding.to_frame().iloc[:,0:0]
top9

""
country_code
USA
CHN
GBR
IND
CAN
FRA
ISR
DEU
JPN


In [30]:
#Code to get the primary sector of each category list from the category_list column
#split the category_list column using str.split method of pandas, and assign them to the relevant new columns
master_frame['primary sector']=master_frame['category_list'].str.split("|", n = -1, expand = True)[0]
master_frame.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,primary sector
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,Media
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,Apps
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,Curated Web


In [32]:
#Use the mapping file 'mapping.csv' to map each primary sector to one of the eight main sectors
#Read mapping.csv file into a dataframe mapping
mapping = pd.read_csv('mapping.csv')
mapping = mapping[~mapping['category_list'].isnull()]
mapping['category_list'] = mapping['category_list'].str.replace('0','na',regex=True)
mapping["main_sector"]=(mapping ==1).idxmax(axis=1)[0:]
mapping=mapping.drop(['Automotive & Sports','Blanks','Cleantech / Semiconductors','Entertainment','Health','Manufacturing','News, Search and Messaging','Others','Social, Finance, Analytics, Advertising'],axis=1)

print(mapping.shape)
mapping.head()

#Get the primary sector series from above and merge with mapping DF to get merged DFof primary sector with main sector



(687, 2)


,category_list,main_sector
1,3D,Manufacturing
2,3D Printing,Manufacturing
3,3D Technology,Manufacturing
4,Accounting,"Social, Finance, Analytics, Advertising"
5,Active Lifestyle,Health


In [33]:
mapping['category_list']=mapping['category_list'].str.lower()
master_frame['primary sector']=master_frame['primary sector'].str.lower()



In [34]:
#Table 5.1
#D1- US, D2-UK, D3-IND
#1. All the columns of the master_frame along with the primary sector and the main sector
merged_df_withmainsector=pd.merge(master_frame,mapping,how='left',left_on='primary sector',right_on='category_list')

merged_df_withmainsector.head()
merged_df_withmainsector.loc[merged_df_withmainsector['primary sector']=="cannabis"].head()## verification of merge table data






,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,status,country_code,state_code,region,city,founded_at,primary sector,category_list_y,main_sector
5210,/organization/american-cannabis-company,/funding-round/2f25888e8a94ceb9bb1ff241520dbe5c,venture,A,01-05-2014,NaN,/organization/american-cannabis-company,American Cannabis Company,http://www.americancannabisconsulting.com/,Cannabis|Venture Capital,ipo,USA,CO,Denver,Denver,01-01-2013,cannabis,cannabis,Health
16024,/organization/c-c-shop-llc,/funding-round/db2e138070141f47ccd4f921d233cb65,venture,NaN,08-05-2015,226800.0,/organization/c-c-shop-llc,C & C CANNABIS COMPANY,http://www.cc-shops.com,Cannabis,operating,USA,WA,Seattle,Seattle,07-01-2013,cannabis,cannabis,Health
16681,/organization/cannabuild,/funding-round/08451ea044904d95b8049676625b77c4,seed,NaN,12-06-2014,225000.0,/organization/cannabuild,CannaBuild,http://cannabuild.me,Cannabis|Healthcare Services|Startups,operating,USA,CO,Denver,Denver,01-02-2014,cannabis,cannabis,Health
21438,/organization/coastal-compassion,/funding-round/6f76d8bb1c6d8329738f84e2e0326a24,seed,NaN,01-11-2013,NaN,/organization/coastal-compassion,Coastal Compassion,http://coastalcompassion.org/,Cannabis,operating,USA,MA,New Bedford,New Bedford,01-01-2012,cannabis,cannabis,Health
21439,/organization/coastal-compassion,/funding-round/7cea2bec26f4bb42f7fa80ec9ab4bd88,venture,B,01-11-2014,NaN,/organization/coastal-compassion,Coastal Compassion,http://coastalcompassion.org/,Cannabis,operating,USA,MA,New Bedford,New Bedford,01-01-2012,cannabis,cannabis,Health


In [48]:
#2. D1 for 1st country USA and funding type venture
D1 = merged_df_withmainsector[merged_df_withmainsector['country_code'].apply(lambda x: x=='USA')&merged_df_withmainsector['funding_round_type'].apply(lambda x: x=='venture')&merged_df_withmainsector['raised_amount_usd'].apply(lambda x: (x>=5000000 and x<=15000000))]
print(D1.shape) 
D1.head()

D1.groupby(['main_sector'])["company_permalink"].count().reset_index(name="count").sort_values(by='count', ascending=False)


(12150, 19)


,main_sector,count
6,Others,2950
7,"Social, Finance, Analytics, Advertising",2714
1,Cleantech / Semiconductors,2350
5,"News, Search and Messaging",1583
3,Health,909
4,Manufacturing,799
2,Entertainment,591
0,Automotive & Sports,167


In [36]:
C1_amount_persector= D1.groupby('main_sector')["raised_amount_usd"].sum().reset_index(name="Total Amount Invested")
C1_amount_persector.sort_values(by=['Total Amount Invested'], ascending = False, inplace=True)
C1_amount_persector

,main_sector,Total Amount Invested
6,Others,2.632101e+10
7,"Social, Finance, Analytics, Advertising",2.380738e+10
1,Cleantech / Semiconductors,2.163343e+10
5,"News, Search and Messaging",1.397157e+10
3,Health,8.211859e+09
4,Manufacturing,7.258553e+09
2,Entertainment,5.099198e+09
0,Automotive & Sports,1.454104e+09


In [37]:
print(C1_amount_persector['Total Amount Invested'].sum())

107757097294.0


In [38]:
#(top sector count-wise), which company received the highest investment?- Others sector
D1.loc[D1.main_sector.apply(lambda x:x=='Others')].groupby(['name']).raised_amount_usd.sum().sort_values(ascending=False).head(1)

name
Virtustream    64300000.0
Name: raised_amount_usd, dtype: float64

In [39]:
#(second best sector count-wise), which company received the highest investment?
D1.loc[D1.main_sector.apply(lambda x:x=='Social, Finance, Analytics, Advertising')].groupby(['name']).raised_amount_usd.sum().sort_values(ascending=False).head(1)

name
SST Inc. (Formerly ShotSpotter)    67933006.0
Name: raised_amount_usd, dtype: float64

In [47]:
D2 = merged_df_withmainsector[ merged_df_withmainsector['country_code'].apply(lambda x: x=='GBR') & merged_df_withmainsector['funding_round_type'].apply(lambda x: x=='venture') & merged_df_withmainsector['raised_amount_usd'].apply(lambda x: (x>=5000000 and x<=15000000))]
print(D2.shape) 
D2.head()

D2.groupby(['main_sector'])["company_permalink"].count().reset_index(name="count").sort_values(by='count', ascending=False)



(628, 19)


,main_sector,count
6,Others,147
7,"Social, Finance, Analytics, Advertising",133
1,Cleantech / Semiconductors,130
5,"News, Search and Messaging",73
2,Entertainment,56
4,Manufacturing,42
3,Health,24
0,Automotive & Sports,16


In [49]:
#(top sector count-wise), which company received the highest investment?- Others sector
D2.loc[D2.main_sector.apply(lambda x:x=='Others')].groupby(['name']).raised_amount_usd.sum().sort_values(ascending=False).head(1)

name
Electric Cloud    37000000.0
Name: raised_amount_usd, dtype: float64

In [50]:
#(second best sector count-wise), which company received the highest investment?
D2.loc[D2.main_sector.apply(lambda x:x=='Social, Finance, Analytics, Advertising')].groupby(['name']).raised_amount_usd.sum().sort_values(ascending=False).head(1)

name
Celltick Technologies    37500000.0
Name: raised_amount_usd, dtype: float64

In [51]:
C2_amount_persector=D2.groupby('main_sector')["raised_amount_usd"].sum().reset_index(name="Total Amount Invested")
C2_amount_persector.sort_values(by=['Total Amount Invested'], ascending = False, inplace=True)
C2_amount_persector

,main_sector,Total Amount Invested
6,Others,1.283624e+09
1,Cleantech / Semiconductors,1.163990e+09
7,"Social, Finance, Analytics, Advertising",1.089404e+09
5,"News, Search and Messaging",6.157462e+08
2,Entertainment,4.827847e+08
4,Manufacturing,3.619403e+08
3,Health,2.145375e+08
0,Automotive & Sports,1.670516e+08


In [43]:
print(C2_amount_persector['Total Amount Invested'].sum())

5379078691.0


In [53]:
#2. D2 for 1st country UK and funding type venture
D3 = merged_df_withmainsector[merged_df_withmainsector['country_code'].apply(lambda x: x=='IND')&merged_df_withmainsector['funding_round_type'].apply(lambda x: x=='venture')&merged_df_withmainsector['raised_amount_usd'].apply(lambda x: (x>=5000000 and x<=15000000))]
print(D3.shape) 
D3.head()
D3.groupby('main_sector')["company_permalink"].count().reset_index(name="count").sort_values(by='count', ascending=False)

(330, 19)


,main_sector,count
6,Others,110
7,"Social, Finance, Analytics, Advertising",60
5,"News, Search and Messaging",52
2,Entertainment,33
4,Manufacturing,21
1,Cleantech / Semiconductors,20
3,Health,19
0,Automotive & Sports,13


In [56]:
#(top sector count-wise), which company received the highest investment?- Others sector
D3.loc[D3.main_sector.apply(lambda x:x=='Others')].groupby(['name']).raised_amount_usd.sum().sort_values(ascending=False).head(1)

name
FirstCry.com    39000000.0
Name: raised_amount_usd, dtype: float64

In [58]:
#(top sector count-wise), which company received the highest investment?- News, Search and Messaging sector
D3.loc[D3.main_sector.apply(lambda x:x=='Social, Finance, Analytics, Advertising')].groupby(['name']).raised_amount_usd.sum().sort_values(ascending=False).head(1)

name
Manthan Systems    50700000.0
Name: raised_amount_usd, dtype: float64

In [54]:
C3_amount_persector = D3.groupby('main_sector')["raised_amount_usd"].sum().reset_index(name="Total Amount Invested")
C3_amount_persector.sort_values(by=['Total Amount Invested'], ascending = False, inplace=True)
C3_amount_persector

,main_sector,Total Amount Invested
6,Others,1.013410e+09
7,"Social, Finance, Analytics, Advertising",5.505496e+08
5,"News, Search and Messaging",4.338345e+08
2,Entertainment,2.808300e+08
4,Manufacturing,2.009000e+08
3,Health,1.677400e+08
1,Cleantech / Semiconductors,1.653800e+08
0,Automotive & Sports,1.369000e+08


In [55]:
print(C3_amount_persector['Total Amount Invested'].sum())

2949543602.0
